### Notebook Overview

In this notebook, these tasks are summarized below:

1. Extracted store sub-categories combines T5 model with manual rule-based classification
2. Created 4 more new columns for analysis
3. Reorganized and saved the cleaned data to CSV files

---

In [1]:
import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns   
import plotly.express as px
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [2]:
df = pd.read_csv('customers_df.csv')

poten_df = pd.read_csv('potential_customers_df.csv')

### Create sub category

Explain: The code combines manual rule-based classification with the T5 model

The reason for combining both is that the T5 model handles more ambiguous cases, and rule-based classification quickly and accurately handles straightforward cases based on predefined keywords

By combining these approaches, the aim is to improve classification accuracy and robustness

In [3]:
descriptions_1 = df['description'].tolist()
descriptions_2 = poten_df['description'].tolist()
descriptions = descriptions_1 + descriptions_2

# smaller case
descriptions = [str(desc).lower() for desc in descriptions]

In [4]:
# Load the T5 model and tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small", model_max_length=512)
model = T5ForConditionalGeneration.from_pretrained("t5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
categories = {
    'bar': ['hub', 'jazz', 'crafted', 'craft','bar'],
    'pub': ['pub', 'irish'],
    'sport bar': ['sports'],
    'japanese restaurant': ['japansese'], #there is typo in the description
    'italian restaurant': ['pasta', 'pizza','italian'],
    'french restaurant': ['french'],
    'spanish restaurant': ['paella', 'small plates'],
    'indian restaurant': ['curries', 'naan', 'mumbai'],
    'chinese restaurant': ['dim sum', 'szechuan'],
    'fusion restaurant': ['east meets west'],
    'vegetarian restaurant': ['plant-based'],
    'budget hotel': ['affordable price', 'thrifty traveler', 'affordable rates', 'no-frills'],
    'eco-friendly hotel': ['sustainable retreat', 'eco-friendly'],
    'boutique hotel': ['cozy retreat', 'boutique-inspired', 'boutique hotel'],
    'urban hotel': ['city center', 'urban'],
    'luxury hotel': ['luxurious', 'luxurious haven', 'luxury retreat', 'luxury hotel'],
    'per-friendly hotel': ['pet', 'furry'],
    'steakhouse': ['steakhouse','meat']
}

In [6]:
def classify_with_rules(description):
    description_lower = description.lower()
    for category, keywords in categories.items():
        for keyword in keywords:
            if keyword in description_lower:
                return category
    return None


In [7]:
def ask_question_abstractive(question, context):
    input_text = f"answer: {question} context: {context}"
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
    outputs = model.generate(inputs["input_ids"], max_length=128, num_return_sequences=1)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

In [8]:
sub_categories = []

for description in descriptions:
    rule_based_category = classify_with_rules(description)
    if rule_based_category:
        sub_categories.append(rule_based_category)
    else:
        question = "What is the store sub-category?"
        answer = ask_question_abstractive(question, description)
        sub_categories.append(answer)

In [9]:
# Split the sub-categories back to the respective dataframes
sub_categories_1 = sub_categories[:len(descriptions_1)]
sub_categories_2 = sub_categories[len(descriptions_1):]

In [10]:
df['store_sub_category'] = sub_categories_1
df['store_sub_category'] = df['store_sub_category'].str.lower()
df['store_sub_category'].value_counts()

store_sub_category
luxury hotel                1661
bar                         1435
boutique hotel               895
pub                          549
french restaurant            364
japanese restaurant          360
steakhouse                   358
chinese restaurant           356
mediterranean restaurant     352
italian restaurant           348
fusion restaurant            343
greek restaurant             342
spanish restaurant           336
mexican restaurant           335
bangkok                      333
indian restaurant            308
urban hotel                  288
budget hotel                 283
eco-friendly hotel           263
sport bar                    130
Name: count, dtype: int64

In [11]:
poten_df['store_sub_category'] = sub_categories_2
poten_df['store_sub_category'] = poten_df['store_sub_category'].str.lower()
poten_df['store_sub_category'].value_counts()

store_sub_category
luxury hotel                5681
bar                         5140
boutique hotel              3443
indian restaurant           2759
urban hotel                 2238
pub                         2171
greek restaurant            1442
fusion restaurant           1406
steakhouse                  1395
chinese restaurant          1367
spanish dishes              1366
italian restaurant          1351
mexican restaurant          1332
sushi rolls                 1331
french restaurant           1322
vegetarian restaurant       1288
greece, italy, and spain    1282
eco-friendly hotel          1145
budget hotel                1112
Name: count, dtype: int64

#### Fix the category that is wrongly categorized

1. incorrect category in df

In [12]:
bangkok = df[df['store_sub_category'] == 'bangkok']
print(bangkok['description'].unique())

['Warm aromas of lemongrass and galangal waft through the doors of this cozy Thai restaurant, where vibrant colors and intricate carvings transport diners to the bustling streets of Bangkok.']


In [13]:
# look into the description of spanish restaurant

spanish_restaurant = df[df['store_sub_category'] == 'per-friendly hotel']
print(spanish_restaurant['description'].unique())

[]


In [14]:
mix_country = poten_df[poten_df['store_sub_category'] == 'greece, italy, and spain']

print(mix_country['description'].unique())

['Vibrant Mediterranean restaurant with warm, golden hues and rustic wooden accents, serving authentic flavors of Greece, Italy, and Spain in cozy, intimate ambiance.']


In [15]:
def update_store_sub_category(dataframe):
    dataframe.loc[dataframe['store_sub_category'] == 'medley of small plates', 'store_sub_category'] = 'spanish restaurant'
    dataframe.loc[dataframe['store_sub_category'] == 'bangkok', 'store_sub_category'] = 'thai restaurant'
    dataframe.loc[dataframe['description'].str.contains('urban', na=False), 'store_sub_category'] = 'urban hotel'
    dataframe.loc[dataframe['description'].str.contains('tapas bar', na=False), 'store_sub_category'] = 'bar'
    dataframe.loc[dataframe['description'].str.contains('plant-based', na=False), 'store_sub_category'] = 'vegetarian restaurant'
    dataframe.loc[dataframe['description'].str.contains('pub', na=False), 'store_sub_category'] = 'pub'
    dataframe.loc[dataframe['description'].str.contains('furry', na=False), 'store_sub_category'] = 'pet-friendly hotel'
    dataframe.loc[dataframe['store_sub_category'] == 'bella vita', 'store_sub_category'] = 'italian restaurant'
    dataframe.loc[dataframe['store_sub_category'] == 'sushi rolls', 'store_sub_category'] = 'japanese restaurant'
    dataframe.loc[dataframe['store_sub_category'] == 'greece, italy, and spain', 'store_sub_category'] = 'mediterranean restaurant'
    dataframe.loc[dataframe['store_sub_category'] == 'spanish dishes', 'store_sub_category'] = 'spanish restaurant'
    
    return dataframe

In [16]:
# Apply the function to df and poten_df
df = update_store_sub_category(df)
poten_df = update_store_sub_category(poten_df)

In [17]:
print(df['store_sub_category'].isnull().sum())
print(poten_df['store_sub_category'].isnull().sum())

0
0


### Create main category

In [18]:
def main_category(store_sub_category):
    if 'hotel' in store_sub_category or 'resort' in store_sub_category:
        return 'hotel'
    elif 'restaurant' in store_sub_category or 'steakhouse' in store_sub_category:
        return 'restaurant'
    elif 'bar' in store_sub_category or 'pub' in store_sub_category:
        return 'bar'
    elif 'pub' in store_sub_category:
        return 'Pub'
    else:
        return np.NAN

In [19]:
df['store_category'] = df['store_sub_category'].apply(main_category)
poten_df['store_category'] = poten_df['store_sub_category'].apply(main_category)

In [20]:
print(df['store_category'].isnull().sum())
print(poten_df['store_category'].isnull().sum())

0
0


In [21]:
df['store_category'].value_counts() / df['store_category'].value_counts().sum() * 100

store_category
restaurant    46.093993
hotel         35.169623
bar           18.736383
Name: count, dtype: float64

In [22]:
df['store_sub_category'].value_counts() / df['store_sub_category'].value_counts().sum() * 100

store_sub_category
luxury hotel                17.232078
bar                         10.478265
pub                          6.909430
urban hotel                  6.100218
french restaurant            3.776325
japanese restaurant          3.734827
steakhouse                   3.714078
chinese restaurant           3.693329
mediterranean restaurant     3.651831
italian restaurant           3.610333
fusion restaurant            3.558460
greek restaurant             3.548086
spanish restaurant           3.485839
mexican restaurant           3.475464
thai restaurant              3.454715
vegetarian restaurant        3.195352
indian restaurant            3.195352
pet-friendly hotel           3.133105
boutique hotel               3.039734
budget hotel                 2.935989
eco-friendly hotel           2.728499
sport bar                    1.348688
Name: count, dtype: float64

### Create "sales_per_employee" column

In [23]:
df['sales_per_employee'] = df['sales_amount'] / df['number_of_employees']

### Create "opening_duration" column

In [24]:
def calculate_opening_duration(row):
    if row['opening_hour'] == row['closing_hour'] and row['opening_hour'] == 0:
        return 24.0
    elif row['closing_hour'] < row['opening_hour']:
        return (24 - row['opening_hour']) + row['closing_hour']
    else:
        return row['closing_hour'] - row['opening_hour']

In [25]:
# Apply the function to calculate the opening duration
df['opening_duration'] = df.apply(calculate_opening_duration, axis=1)
poten_df['opening_duration'] = poten_df.apply(calculate_opening_duration, axis=1)

In [26]:
df['opening_duration'].value_counts()

opening_duration
12.0    6358
24.0    1161
17.0    1158
7.0      487
13.0     475
Name: count, dtype: int64

In [27]:
poten_df['opening_duration'].value_counts()

opening_duration
12.0    25636
24.0     4555
17.0     4463
13.0     1961
7.0      1956
Name: count, dtype: int64

### Create "operation_time" column

In [28]:
def operation_time(hour):
    if 0 <= hour < 6:
        return 'Night'
    elif 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    else:
        return 'Evening'

df['opening_timeofday'] = df['opening_hour'].apply(operation_time)
poten_df['opening_timeofday'] = poten_df['opening_hour'].apply(operation_time)

df['closing_timeofday'] = df['closing_hour'].apply(operation_time)
poten_df['closing_timeofday'] = poten_df['closing_hour'].apply(operation_time)

### Create "young_pop_percentage" column

In [29]:
def young_pop_percentage(row):
    return row['young_population'] / row['population'] * 100

df['young_pop_percentage'] = df.apply(young_pop_percentage, axis=1)
poten_df['young_pop_percentage'] = poten_df.apply(young_pop_percentage, axis=1)

### Create "sales_rep_pop" & "store_manage_by_rep" column

See the percentage of the population managed by each sales representative based on the number of unique stores they oversee

In [31]:
store_manage = df.groupby('sales_representative_id')['id'].nunique().reset_index()
store_manage.columns = ['sales_representative_id', 'store_manage']

df = df.merge(store_manage, on='sales_representative_id', how='left')

df['sales_rep_pop'] = df['store_manage'] / df['population'] * 100

### Reorganized column and save to csv

In [32]:
## reorganized the column and save it
reordered_columns = [
    # Store Information
    'id', 'store_category', 'store_sub_category', 'description',
    # Location
    'city', 'latitude', 'longitude', 'distance',
    # Operational Details
    'store_size', 'opening_hour', 'closing_hour', 'opening_duration', 'date_opening', 'opening_timeofday', 'closing_timeofday' ,'parking',
    # Demographics
    'population', 'young_population', 'young_pop_percentage' ,'gdb_per_capita', 'unemployment_rate',
    # Reviews and Ratings
    'number_of_reviews', 'rating', 'rating_round',
    # Employees
    'number_of_employees', 'sales_representative_id', 'sales_representative_latitude', 'sales_representative_longitude','sales_rep_pop', 
    # Financials
    'sales_amount', 'sales_per_employee'
]

In [33]:
# Reorganize the DataFrame
df = df[reordered_columns]

In [34]:
# save to CSV
df.to_csv('customers_df.csv', index=False)

In [35]:
 
reordered_columns = [
    # Store Information
    'id', 'store_category', 'store_sub_category', 'description',
    # Location
    'city', 'latitude', 'longitude', 'distance',
    # Operational Details
    'store_size', 'opening_hour', 'closing_hour', 'opening_duration', 'date_opening', 'opening_timeofday', 'closing_timeofday' , 'parking',
    # Demographics
    'population', 'young_population', 'young_pop_percentage', 'gdb_per_capita', 'unemployment_rate',
    # Reviews and Ratings
    'number_of_reviews', 'rating', 'rating_round',
    # Employees
    'number_of_employees'
]

In [36]:
# Reorganize the DataFrame
poential_df = poten_df[reordered_columns]

In [37]:
# save to CSV

poential_df.to_csv('potential_customers_df.csv', index=False)